# Rent vs. Buy Calculator: Complete Calculation Tutorial

This notebook walks through the entire calculation logic used in **ownvsrent.io** to compare renting versus buying a home. We'll explain each step, show the formulas, and work through a concrete example.

## Table of Contents
1. [Overview & Key Principles](#1-overview--key-principles)
2. [Input Parameters](#2-input-parameters)
3. [Mortgage Amortization](#3-mortgage-amortization)
4. [Renter Cash Flows](#4-renter-cash-flows)
5. [Buyer Cash Flows](#5-buyer-cash-flows)
6. [PMI Calculation & Removal](#6-pmi-calculation--removal)
7. [Tax Benefits (The Critical Part)](#7-tax-benefits-the-critical-part)
8. [Portfolio Growth](#8-portfolio-growth)
9. [Final Wealth Comparison](#9-final-wealth-comparison)
10. [Worked Example](#10-worked-example)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set display options
pd.set_option('display.float_format', lambda x: '${:,.0f}'.format(x) if abs(x) >= 1 else '{:.4f}'.format(x))
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Overview & Key Principles

### The Core Question
Given the same amount of money, which path leaves you **wealthier** after N years?

- **Renter**: Invests down payment + closing costs into the stock market, invests monthly savings when rent < ownership cost
- **Buyer**: Builds equity through mortgage payments + appreciation, invests monthly savings when ownership < rent

### Key Principles

1. **Month-by-month simulation** - We don't use annual approximations. The engine iterates monthly for accuracy.

2. **Fair comparison** - Both parties invest their surplus. If renting is cheaper one month, the renter invests the difference. If buying is cheaper, the buyer invests the difference.

3. **Tax benefit = ONLY the excess over standard deduction** - This is the #1 mistake in competing calculators. Most people don't itemize, so mortgage interest provides $0 tax benefit.

4. **All costs included** - We account for PMI, maintenance, selling costs, capital gains taxes, etc.

## 2. Input Parameters

Let's define our example scenario with default values:

In [ ]:
# === EXAMPLE INPUTS ===

# Renting
monthly_rent = 2000
annual_rent_increase = 0.03       # 3% per year
renter_insurance = 30              # per month
security_deposit_months = 1        # months of rent
broker_fee = 0                     # one-time, as decimal of annual rent

# Buying
purchase_price = 400_000
down_payment_percent = 0.20        # 20%
mortgage_rate = 0.068              # 6.8% annual
loan_term_years = 30
property_tax_rate = 0.011          # 1.1% of home value
home_insurance_rate = 0.005        # 0.5% of home value
hoa_monthly = 0
maintenance_rate = 0.015           # 1.5% of home value
pmi_rate = 0.0075                  # 0.75% of loan
closing_costs_percent = 0.03       # 3% of loan
selling_costs_percent = 0.08       # 8% of sale price

# Financial
holding_period_years = 7
annual_appreciation = 0.035        # 3.5%
annual_investment_return = 0.07    # 7%
marginal_tax_rate = 0.22           # 22% federal bracket
state_tax_rate = 0.05              # 5%
filing_status = 'single'
capital_gains_tax_rate = 0.15      # 15%

# Derived values
down_payment = purchase_price * down_payment_percent
loan_amount = purchase_price - down_payment
closing_costs = loan_amount * closing_costs_percent

print(f"Purchase Price: ${purchase_price:,.0f}")
print(f"Down Payment (20%): ${down_payment:,.0f}")
print(f"Loan Amount: ${loan_amount:,.0f}")
print(f"Closing Costs (3%): ${closing_costs:,.0f}")

## 3. Mortgage Amortization

### Monthly Payment Formula

The standard amortization formula for a fixed-rate mortgage:

$$M = P \times \frac{r(1+r)^n}{(1+r)^n - 1}$$

Where:
- $M$ = Monthly payment
- $P$ = Principal (loan amount)
- $r$ = Monthly interest rate (annual rate / 12)
- $n$ = Total number of payments (years × 12)

In [ ]:
def calculate_monthly_payment(principal, annual_rate, term_years):
    """Calculate fixed monthly mortgage payment (P&I only)."""
    if annual_rate == 0:
        return principal / (term_years * 12)
    
    monthly_rate = annual_rate / 12
    num_payments = term_years * 12
    
    payment = principal * (
        (monthly_rate * (1 + monthly_rate) ** num_payments)
        / ((1 + monthly_rate) ** num_payments - 1)
    )
    return payment

monthly_payment = calculate_monthly_payment(loan_amount, mortgage_rate, loan_term_years)
print(f"Monthly P&I Payment: ${monthly_payment:,.2f}")
print(f"Total Payments over {loan_term_years} years: ${monthly_payment * loan_term_years * 12:,.0f}")
print(f"Total Interest Paid: ${monthly_payment * loan_term_years * 12 - loan_amount:,.0f}")

### Loan Balance Formula

Remaining balance after $p$ payments:

$$B_p = P \times \frac{(1+r)^n - (1+r)^p}{(1+r)^n - 1}$$

In [ ]:
def calculate_loan_balance(principal, annual_rate, term_years, month):
    """Calculate remaining loan balance after N months."""
    if month <= 0:
        return principal
    if annual_rate == 0:
        monthly_payment = principal / (term_years * 12)
        return max(0, principal - monthly_payment * month)
    
    monthly_rate = annual_rate / 12
    num_payments = term_years * 12
    
    if month >= num_payments:
        return 0
    
    balance = principal * (
        ((1 + monthly_rate) ** num_payments - (1 + monthly_rate) ** month)
        / ((1 + monthly_rate) ** num_payments - 1)
    )
    return max(0, balance)

# Show balance at key milestones
for year in [1, 5, 7, 10, 15, 20, 25, 30]:
    balance = calculate_loan_balance(loan_amount, mortgage_rate, loan_term_years, year * 12)
    print(f"Year {year:2d}: ${balance:>12,.0f} remaining ({balance/loan_amount*100:.1f}%)")

### Principal vs Interest Breakdown

Each payment splits between interest (on current balance) and principal:

In [ ]:
def calculate_payment_breakdown(principal, annual_rate, term_years, month):
    """Calculate principal and interest portions for a specific month."""
    if month <= 0:
        return 0, 0
    
    payment = calculate_monthly_payment(principal, annual_rate, term_years)
    
    if annual_rate == 0:
        return payment, 0
    
    balance_before = calculate_loan_balance(principal, annual_rate, term_years, month - 1)
    monthly_rate = annual_rate / 12
    interest_portion = balance_before * monthly_rate
    principal_portion = payment - interest_portion
    
    return principal_portion, interest_portion

# Visualize amortization
months = range(1, loan_term_years * 12 + 1)
principals = []
interests = []

for m in months:
    p, i = calculate_payment_breakdown(loan_amount, mortgage_rate, loan_term_years, m)
    principals.append(p)
    interests.append(i)

plt.figure(figsize=(12, 5))
plt.stackplot(months, interests, principals, labels=['Interest', 'Principal'], 
              colors=['#ef4444', '#22c55e'], alpha=0.8)
plt.xlabel('Month')
plt.ylabel('Payment ($)')
plt.title('Mortgage Payment Breakdown Over Time')
plt.legend(loc='upper right')
plt.axhline(y=monthly_payment, color='black', linestyle='--', alpha=0.5, label='Total Payment')
plt.show()

# Show first year breakdown
year1_interest = sum(interests[:12])
year1_principal = sum(principals[:12])
print(f"\nYear 1 Breakdown:")
print(f"  Total Payments: ${year1_interest + year1_principal:,.0f}")
print(f"  Interest Paid: ${year1_interest:,.0f} ({year1_interest/(year1_interest+year1_principal)*100:.1f}%)")
print(f"  Principal Paid: ${year1_principal:,.0f} ({year1_principal/(year1_interest+year1_principal)*100:.1f}%)")

## 4. Renter Cash Flows

### Monthly Cost
Rent + Renter's Insurance. Rent increases annually.

In [ ]:
def calculate_monthly_rent(base_rent, month, annual_increase):
    """Rent with annual escalation."""
    years_elapsed = (month - 1) // 12
    return base_rent * (1 + annual_increase) ** years_elapsed

# Show rent progression
for year in range(1, holding_period_years + 1):
    month = year * 12
    rent = calculate_monthly_rent(monthly_rent, month, annual_rent_increase)
    total_cost = rent + renter_insurance
    print(f"Year {year}: Rent ${rent:,.0f}/mo + Insurance ${renter_insurance}/mo = ${total_cost:,.0f}/mo")

### Renter's Initial Investment

The renter invests what they would have spent on:
- Down payment (buyer's)
- Closing costs (buyer's)

Minus their own costs:
- Broker fee (if any)

In [ ]:
renter_initial_investment = down_payment + closing_costs - (monthly_rent * 12 * broker_fee)
print(f"Renter's Initial Investment: ${renter_initial_investment:,.0f}")
print(f"  = Down Payment (${down_payment:,.0f}) + Closing Costs (${closing_costs:,.0f})")

## 5. Buyer Cash Flows

### Home Value with Appreciation

In [ ]:
def calculate_home_value(purchase_price, month, annual_appreciation):
    """Home value with monthly compounding."""
    monthly_rate = (1 + annual_appreciation) ** (1 / 12) - 1
    return purchase_price * (1 + monthly_rate) ** month

# Show home value progression
for year in range(1, holding_period_years + 1):
    value = calculate_home_value(purchase_price, year * 12, annual_appreciation)
    gain = value - purchase_price
    print(f"Year {year}: ${value:>12,.0f} (↑${gain:>9,.0f}, {gain/purchase_price*100:+.1f}%)")

### Monthly Ownership Costs (PITI + HOA + Maintenance + PMI)

In [ ]:
def calculate_buyer_monthly_cost(month, purchase_price, loan_amount, mortgage_payment,
                                  annual_appreciation, property_tax_rate, home_insurance_rate,
                                  maintenance_rate, hoa_monthly, pmi_rate, down_payment_pct):
    """Calculate total monthly cost for buyer."""
    home_value = calculate_home_value(purchase_price, month, annual_appreciation)
    loan_balance = calculate_loan_balance(loan_amount, mortgage_rate, loan_term_years, month)
    
    # Property tax (based on current value)
    property_tax = (home_value * property_tax_rate) / 12
    
    # Home insurance (based on current value)
    home_insurance = (home_value * home_insurance_rate) / 12
    
    # Maintenance (based on current value)
    maintenance = (home_value * maintenance_rate) / 12
    
    # PMI (removed when LTV <= 80%)
    ltv = loan_balance / home_value
    if down_payment_pct >= 0.20 or ltv <= 0.80:
        pmi = 0
    else:
        pmi = (loan_amount * pmi_rate) / 12
    
    total = mortgage_payment + property_tax + home_insurance + maintenance + hoa_monthly + pmi
    
    return {
        'mortgage': mortgage_payment,
        'property_tax': property_tax,
        'home_insurance': home_insurance,
        'maintenance': maintenance,
        'hoa': hoa_monthly,
        'pmi': pmi,
        'total': total
    }

# Show month 1 breakdown
month1 = calculate_buyer_monthly_cost(1, purchase_price, loan_amount, monthly_payment,
                                       annual_appreciation, property_tax_rate, home_insurance_rate,
                                       maintenance_rate, hoa_monthly, pmi_rate, down_payment_percent)

print("Month 1 Ownership Cost Breakdown:")
print(f"  Mortgage (P&I):  ${month1['mortgage']:>8,.0f}")
print(f"  Property Tax:    ${month1['property_tax']:>8,.0f}")
print(f"  Home Insurance:  ${month1['home_insurance']:>8,.0f}")
print(f"  Maintenance:     ${month1['maintenance']:>8,.0f}")
print(f"  HOA:             ${month1['hoa']:>8,.0f}")
print(f"  PMI:             ${month1['pmi']:>8,.0f}")
print(f"  ─────────────────────────")
print(f"  TOTAL:           ${month1['total']:>8,.0f}/month")
print(f"\n  vs. Rent:        ${monthly_rent + renter_insurance:>8,.0f}/month")
print(f"  Difference:      ${month1['total'] - (monthly_rent + renter_insurance):>+8,.0f}/month")

## 6. PMI Calculation & Removal

Private Mortgage Insurance (PMI) is required when down payment < 20%.

**Key insight**: PMI is removed when Loan-to-Value (LTV) drops to 80%, based on **current appraised value** (not original purchase price). This means appreciation helps you escape PMI faster!

In [ ]:
def find_pmi_removal_month(loan_amount, purchase_price, mortgage_rate, 
                           loan_term_years, annual_appreciation, down_payment_pct):
    """Find when PMI is removed (LTV <= 80% of current value)."""
    if down_payment_pct >= 0.20:
        return None  # No PMI needed
    
    for month in range(1, loan_term_years * 12 + 1):
        balance = calculate_loan_balance(loan_amount, mortgage_rate, loan_term_years, month)
        home_value = calculate_home_value(purchase_price, month, annual_appreciation)
        ltv = balance / home_value
        
        if ltv <= 0.80:
            return month
    
    return None

# Example with 10% down payment
low_down_pct = 0.10
low_down = purchase_price * low_down_pct
low_loan = purchase_price - low_down

pmi_month = find_pmi_removal_month(low_loan, purchase_price, mortgage_rate,
                                    loan_term_years, annual_appreciation, low_down_pct)

if pmi_month:
    balance_at_removal = calculate_loan_balance(low_loan, mortgage_rate, loan_term_years, pmi_month)
    value_at_removal = calculate_home_value(purchase_price, pmi_month, annual_appreciation)
    print(f"With 10% down payment (${low_down:,.0f}):")
    print(f"  PMI removed at month {pmi_month} (year {pmi_month/12:.1f})")
    print(f"  Home value at removal: ${value_at_removal:,.0f}")
    print(f"  Loan balance at removal: ${balance_at_removal:,.0f}")
    print(f"  LTV at removal: {balance_at_removal/value_at_removal*100:.1f}%")
    print(f"  Monthly PMI savings: ${low_loan * pmi_rate / 12:,.0f}")

## 7. Tax Benefits (The Critical Part)

### The #1 Mistake in Competing Calculators

Most calculators assume you get a tax benefit of:
```
tax_benefit = mortgage_interest × marginal_tax_rate
```

**This is WRONG!**

The correct calculation is:
```
tax_benefit = max(0, itemized_deductions - standard_deduction) × marginal_tax_rate
```

If your itemized deductions (mortgage interest + property tax + state income tax) are less than the standard deduction, **you get ZERO tax benefit from your mortgage**.

### 2025-2026 Tax Law (OBBBA)
- **Standard Deduction**: $15,750 (single) / $31,500 (married)
- **SALT Cap**: $40,000 (2025-2029), then $10,000 (2030+)
- **Mortgage Interest Cap**: Deductible on first $750K of debt only

In [ ]:
# Tax constants
STANDARD_DEDUCTION_SINGLE = 15_750
STANDARD_DEDUCTION_MARRIED = 31_500
SALT_CAP = 40_000  # 2025-2029
MORTGAGE_INTEREST_CAP = 750_000

def calculate_tax_benefit(mortgage_interest, property_tax, state_income_tax, 
                          loan_amount, marginal_rate, filing_status):
    """Calculate ACTUAL tax benefit (excess over standard deduction)."""
    
    # 1. Calculate deductible mortgage interest (cap at $750K loan)
    if loan_amount <= MORTGAGE_INTEREST_CAP:
        deductible_interest = mortgage_interest
    else:
        proration = MORTGAGE_INTEREST_CAP / loan_amount
        deductible_interest = mortgage_interest * proration
    
    # 2. Calculate SALT (capped at $40K)
    total_salt = property_tax + state_income_tax
    deductible_salt = min(total_salt, SALT_CAP)
    
    # 3. Total itemized deductions
    itemized = deductible_interest + deductible_salt
    
    # 4. Standard deduction
    standard = STANDARD_DEDUCTION_MARRIED if filing_status == 'married' else STANDARD_DEDUCTION_SINGLE
    
    # 5. Tax benefit = ONLY THE EXCESS
    if itemized <= standard:
        return 0, False  # No benefit, take standard deduction
    
    excess = itemized - standard
    benefit = excess * marginal_rate
    
    return benefit, True

# Calculate year 1 tax benefit
year1_interest = sum(interests[:12])
year1_prop_tax = calculate_home_value(purchase_price, 12, annual_appreciation) * property_tax_rate
estimated_income = 100_000
year1_state_tax = estimated_income * state_tax_rate

benefit, itemizing = calculate_tax_benefit(
    year1_interest, year1_prop_tax, year1_state_tax,
    loan_amount, marginal_tax_rate, filing_status
)

print("Year 1 Tax Analysis:")
print(f"\n  Mortgage Interest: ${year1_interest:>12,.0f}")
print(f"  Property Tax:      ${year1_prop_tax:>12,.0f}")
print(f"  State Income Tax:  ${year1_state_tax:>12,.0f}")
print(f"  ─────────────────────────────")
print(f"  SALT (capped):     ${min(year1_prop_tax + year1_state_tax, SALT_CAP):>12,.0f}")
print(f"  Total Itemized:    ${year1_interest + min(year1_prop_tax + year1_state_tax, SALT_CAP):>12,.0f}")
print(f"  Standard Deduction: ${STANDARD_DEDUCTION_SINGLE:>11,.0f}")
print(f"\n  Itemizing beneficial? {'YES' if itemizing else 'NO'}")
print(f"  Tax Benefit: ${benefit:,.0f}")

if not itemizing:
    print(f"\n  ⚠️  With standard deduction, mortgage interest provides $0 tax benefit!")

## 8. Portfolio Growth

### Fair Comparison: Both Parties Invest Surpluses

Each month:
- If rent < ownership cost → Renter invests the difference
- If ownership < rent → Buyer invests the difference

Portfolio grows with monthly compounding:

In [ ]:
def grow_portfolio(current_value, monthly_contribution, monthly_return_rate):
    """Grow portfolio for one month."""
    return current_value * (1 + monthly_return_rate) + monthly_contribution

monthly_return = (1 + annual_investment_return) ** (1/12) - 1

# Simulate portfolios
renter_portfolio = renter_initial_investment
buyer_portfolio = 0
renter_history = [renter_portfolio]
buyer_history = [buyer_portfolio]

for month in range(1, holding_period_years * 12 + 1):
    # Renter cost
    rent = calculate_monthly_rent(monthly_rent, month, annual_rent_increase)
    renter_cost = rent + renter_insurance
    
    # Buyer cost
    buyer_costs = calculate_buyer_monthly_cost(
        month, purchase_price, loan_amount, monthly_payment,
        annual_appreciation, property_tax_rate, home_insurance_rate,
        maintenance_rate, hoa_monthly, pmi_rate, down_payment_percent
    )
    buyer_cost = buyer_costs['total']
    
    # Who saves money this month?
    difference = buyer_cost - renter_cost
    
    if difference > 0:  # Renting is cheaper
        renter_contribution = difference
        buyer_contribution = 0
    else:  # Buying is cheaper
        renter_contribution = 0
        buyer_contribution = -difference
    
    renter_portfolio = grow_portfolio(renter_portfolio, renter_contribution, monthly_return)
    buyer_portfolio = grow_portfolio(buyer_portfolio, buyer_contribution, monthly_return)
    
    renter_history.append(renter_portfolio)
    buyer_history.append(buyer_portfolio)

# Plot portfolios
plt.figure(figsize=(10, 5))
months = range(0, holding_period_years * 12 + 1)
plt.plot(months, renter_history, label='Renter Portfolio', color='#0284c7', linewidth=2)
plt.plot(months, buyer_history, label='Buyer Portfolio', color='#059669', linewidth=2)
plt.xlabel('Month')
plt.ylabel('Portfolio Value ($)')
plt.title('Investment Portfolio Growth')
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
plt.show()

print(f"Final Portfolios after {holding_period_years} years:")
print(f"  Renter: ${renter_portfolio:,.0f}")
print(f"  Buyer:  ${buyer_portfolio:,.0f}")

## 9. Final Wealth Comparison

At the end of the holding period, we calculate net wealth for each party:

### Renter's Wealth
```
Renter Wealth = Portfolio (after cap gains tax) + Security Deposit Returned
```

### Buyer's Wealth
```
Buyer Wealth = Home Sale Proceeds + Portfolio (after cap gains tax)

Home Sale Proceeds = Sale Price - Loan Balance - Selling Costs - Capital Gains Tax
```

**Capital Gains Exemption**: $250K (single) or $500K (married) if owned 2+ years.

In [ ]:
CAP_GAINS_EXEMPTION_SINGLE = 250_000
CAP_GAINS_EXEMPTION_MARRIED = 500_000

def calculate_final_wealth(holding_years, purchase_price, loan_amount, annual_appreciation,
                           selling_costs_pct, capital_gains_rate, filing_status,
                           renter_portfolio, buyer_portfolio, renter_initial,
                           security_deposit):
    """Calculate final wealth for both parties."""
    
    months = holding_years * 12
    
    # === BUYER ===
    final_home_value = calculate_home_value(purchase_price, months, annual_appreciation)
    final_loan_balance = calculate_loan_balance(loan_amount, mortgage_rate, loan_term_years, months)
    selling_costs = final_home_value * selling_costs_pct
    
    # Capital gains
    home_gain = final_home_value - purchase_price
    if holding_years >= 2:
        exemption = CAP_GAINS_EXEMPTION_MARRIED if filing_status == 'married' else CAP_GAINS_EXEMPTION_SINGLE
        taxable_gain = max(0, home_gain - exemption)
    else:
        taxable_gain = max(0, home_gain)
    cap_gains_tax = taxable_gain * capital_gains_rate
    
    # Net proceeds from sale
    gross_equity = final_home_value - final_loan_balance
    net_proceeds = gross_equity - selling_costs - cap_gains_tax
    
    # Buyer's portfolio after tax
    buyer_portfolio_gain = max(0, buyer_portfolio - 0)  # Contributions are cost basis
    buyer_portfolio_tax = buyer_portfolio_gain * capital_gains_rate
    buyer_portfolio_after_tax = buyer_portfolio - buyer_portfolio_tax
    
    buyer_wealth = net_proceeds + buyer_portfolio_after_tax
    
    # === RENTER ===
    renter_gain = max(0, renter_portfolio - renter_initial)
    renter_portfolio_tax = renter_gain * capital_gains_rate
    renter_portfolio_after_tax = renter_portfolio - renter_portfolio_tax
    
    renter_wealth = renter_portfolio_after_tax + security_deposit
    
    return {
        'buyer': {
            'home_value': final_home_value,
            'loan_balance': final_loan_balance,
            'gross_equity': gross_equity,
            'selling_costs': selling_costs,
            'cap_gains_tax': cap_gains_tax,
            'net_proceeds': net_proceeds,
            'portfolio': buyer_portfolio_after_tax,
            'total_wealth': buyer_wealth
        },
        'renter': {
            'portfolio': renter_portfolio_after_tax,
            'security_deposit': security_deposit,
            'total_wealth': renter_wealth
        },
        'net_benefit': buyer_wealth - renter_wealth,
        'verdict': 'buy' if buyer_wealth - renter_wealth > 1000 else ('rent' if renter_wealth - buyer_wealth > 1000 else 'toss-up')
    }

security_deposit = monthly_rent * security_deposit_months
result = calculate_final_wealth(
    holding_period_years, purchase_price, loan_amount, annual_appreciation,
    selling_costs_percent, capital_gains_tax_rate, filing_status,
    renter_portfolio, buyer_portfolio, renter_initial_investment,
    security_deposit
)

print(f"═══════════════════════════════════════════════════════════")
print(f"FINAL WEALTH COMPARISON (After {holding_period_years} Years)")
print(f"═══════════════════════════════════════════════════════════")
print(f"\nBUYER:")
print(f"  Home Value:       ${result['buyer']['home_value']:>12,.0f}")
print(f"  - Loan Balance:   ${result['buyer']['loan_balance']:>12,.0f}")
print(f"  ─────────────────────────────────")
print(f"  = Gross Equity:   ${result['buyer']['gross_equity']:>12,.0f}")
print(f"  - Selling Costs:  ${result['buyer']['selling_costs']:>12,.0f}")
print(f"  - Cap Gains Tax:  ${result['buyer']['cap_gains_tax']:>12,.0f}")
print(f"  ─────────────────────────────────")
print(f"  = Net Proceeds:   ${result['buyer']['net_proceeds']:>12,.0f}")
print(f"  + Portfolio:      ${result['buyer']['portfolio']:>12,.0f}")
print(f"  ═══════════════════════════════")
print(f"  BUYER WEALTH:     ${result['buyer']['total_wealth']:>12,.0f}")
print(f"\nRENTER:")
print(f"  Portfolio:        ${result['renter']['portfolio']:>12,.0f}")
print(f"  + Security Dep:   ${result['renter']['security_deposit']:>12,.0f}")
print(f"  ═══════════════════════════════")
print(f"  RENTER WEALTH:    ${result['renter']['total_wealth']:>12,.0f}")
print(f"\n{'─'*55}")
print(f"NET BENEFIT: ${result['net_benefit']:>+12,.0f}")
print(f"VERDICT: {result['verdict'].upper()}")

## 10. Worked Example: Sensitivity to Key Variables

Let's see how the verdict changes as we vary key inputs:

In [ ]:
def run_scenario(holding_years, appreciation, investment_return, rent, price):
    """Run a complete scenario and return net benefit."""
    down = price * down_payment_percent
    loan = price - down
    closing = loan * closing_costs_percent
    mortgage = calculate_monthly_payment(loan, mortgage_rate, loan_term_years)
    
    renter_init = down + closing
    renter_port = renter_init
    buyer_port = 0
    monthly_ret = (1 + investment_return) ** (1/12) - 1
    
    for month in range(1, holding_years * 12 + 1):
        r = calculate_monthly_rent(rent, month, annual_rent_increase) + renter_insurance
        
        hv = calculate_home_value(price, month, appreciation)
        lb = calculate_loan_balance(loan, mortgage_rate, loan_term_years, month)
        ltv = lb / hv
        pmi = 0 if down_payment_percent >= 0.2 or ltv <= 0.8 else (loan * pmi_rate / 12)
        
        b = (mortgage + (hv * property_tax_rate / 12) + (hv * home_insurance_rate / 12) + 
             (hv * maintenance_rate / 12) + hoa_monthly + pmi)
        
        diff = b - r
        if diff > 0:
            renter_port = renter_port * (1 + monthly_ret) + diff
            buyer_port = buyer_port * (1 + monthly_ret)
        else:
            renter_port = renter_port * (1 + monthly_ret)
            buyer_port = buyer_port * (1 + monthly_ret) - diff
    
    # Final wealth
    final_hv = calculate_home_value(price, holding_years * 12, appreciation)
    final_lb = calculate_loan_balance(loan, mortgage_rate, loan_term_years, holding_years * 12)
    sell_costs = final_hv * selling_costs_percent
    
    gain = final_hv - price
    exemption = 250_000 if filing_status == 'single' else 500_000
    taxable = max(0, gain - exemption) if holding_years >= 2 else max(0, gain)
    cap_tax = taxable * capital_gains_tax_rate
    
    buyer_wealth = (final_hv - final_lb - sell_costs - cap_tax) + buyer_port * (1 - capital_gains_tax_rate * (buyer_port > 0))
    renter_gain = max(0, renter_port - renter_init)
    renter_wealth = renter_port - renter_gain * capital_gains_tax_rate + rent * security_deposit_months
    
    return buyer_wealth - renter_wealth

# Vary holding period
years_range = range(1, 16)
net_benefits = [run_scenario(y, annual_appreciation, annual_investment_return, monthly_rent, purchase_price) 
                for y in years_range]

plt.figure(figsize=(10, 5))
colors = ['#059669' if nb > 0 else '#0284c7' for nb in net_benefits]
plt.bar(years_range, net_benefits, color=colors, alpha=0.8)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.xlabel('Holding Period (Years)')
plt.ylabel('Net Benefit (+ = Buy Wins)')
plt.title('Buy vs. Rent: Impact of Holding Period')
plt.grid(True, alpha=0.3, axis='y')
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Find break-even
for i, nb in enumerate(net_benefits):
    if nb > 0:
        print(f"Break-even at Year {i+1}")
        break
else:
    print("Renting wins for all holding periods shown")

plt.show()

In [ ]:
# Compare different appreciation rates
appreciation_rates = [0.0, 0.02, 0.035, 0.05, 0.07]

plt.figure(figsize=(10, 5))
for app_rate in appreciation_rates:
    benefits = [run_scenario(y, app_rate, annual_investment_return, monthly_rent, purchase_price) 
                for y in years_range]
    plt.plot(years_range, benefits, label=f'{app_rate*100:.0f}% appreciation', linewidth=2)

plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.xlabel('Holding Period (Years)')
plt.ylabel('Net Benefit (+ = Buy Wins)')
plt.title('Impact of Home Appreciation Rate')
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
plt.show()

## Key Takeaways

1. **Time matters**: Buying typically requires 5-7+ years to break even due to transaction costs (closing + selling costs ~11%).

2. **Tax benefits are often zero**: With the high standard deduction ($15,750 single), most homeowners don't actually benefit from the mortgage interest deduction.

3. **Appreciation is the key variable**: In appreciating markets, buying wins. In flat markets, renting often wins.

4. **The renter invests the difference**: Fair comparison requires the renter to invest the down payment and any monthly savings.

5. **Selling costs are substantial**: 6% commission + 2% closing costs = 8% of home value to sell.

6. **PMI adds cost but goes away**: With appreciation, PMI can be removed faster than expected.

---

This tutorial covers the complete calculation logic used in ownvsrent.io. The actual backend implementation uses month-by-month simulation with all the formulas shown here.